# Reinforcement Learning

## Abstract

In [11]:
# Basic Imports

import numpy as np
import gym
import random
import time
from IPython.display import clear_output

In [12]:
env = gym.make("FrozenLake-v0")

## Baseline Model - Off Policy Q-Learning

### 𝑄 (𝑠,𝑎 )← 𝑄 (𝑠,𝑎 )+𝛼 (𝑟 + 𝛾 max𝑎′𝑄 (𝑠′,𝑎′) − 𝑄(𝑠,𝑎) ) , where 𝑎′ are all actions, that were probed in state 𝑠′.

In [3]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [4]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [5]:
# Number of episodes agent will play during training 
num_episodes = 5000

# Maximum number of steps agent takes per episode
max_steps_per_episode = 100

#Alpha - Learning Rate
learning_rate = 0.7

#Gamma - Discounting Rate
discount_rate = 0.8

#Epsilon - Exploration Rate
exploration_rate = 1.0

max_exploration_rate = 1
min_exploration_rate = 0.01

# Exponential Decay Rate
exploration_decay_rate = 0.01

In [6]:
# Q-learning algorithm

rewards_all_episodes = []
avg_step = []
for episode in range(num_episodes):
    
    # initialize new episode params
    state = env.reset()
    done = False
    rewards_current_episode = 0
    cnt_step = 0
    
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        cnt_step = cnt_step+1 
        
        
        exploration_rate_threshold = random.uniform(0, 1)
        
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()
                
        # Take new action
        new_state, reward, done, info = env.step(action)

        # Update Q-table
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        # Set new state
        state = new_state
        rewards_current_episode += reward 
        
        # Add new reward        
        if done == True: 
            break
    #Appending the average step
    avg_step.append(cnt_step)    
        
    # Exploration rate decay   
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)

# Calculate and print the average reward per thousand episodes
rewards_per_thosand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/500)
count = 500



In [7]:
print("Average Step taken:",int(sum(avg_step)/len(avg_step)))

Average Step taken: 23


In [8]:
print("********Average reward per thousand episodes********\n")
for r in rewards_per_thosand_episodes:
    print(count, ": ", str(sum(r/500)))
    count += 500

********Average reward per thousand episodes********

500 :  0.11000000000000008
1000 :  0.3960000000000003
1500 :  0.2600000000000002
2000 :  0.23600000000000018
2500 :  0.19000000000000014
3000 :  0.3060000000000002
3500 :  0.3040000000000002
4000 :  0.3920000000000003
4500 :  0.3720000000000003
5000 :  0.2740000000000002


In [9]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)    



********Q-table********

[[2.30019471e-04 1.61307959e-03 2.31271883e-03 1.66758174e-03]
 [6.96994883e-04 1.95532216e-04 1.76722562e-03 1.57076801e-03]
 [4.52504048e-02 9.26105056e-05 1.33484041e-03 1.76946772e-03]
 [6.81166040e-04 1.48645933e-05 1.24341525e-05 2.05627645e-03]
 [4.03582378e-03 4.11546464e-05 2.11432848e-04 2.04704168e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [6.70267181e-04 3.19237036e-07 2.38967723e-02 2.63931839e-09]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.91118147e-03 1.91836552e-06 7.87043709e-04 1.04935599e-02]
 [3.99938518e-03 2.74621807e-01 4.51166987e-04 2.15368680e-02]
 [1.49040984e-03 4.25047179e-02 1.21329761e-04 5.62567773e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.23513226e-02 1.33543301e-02 2.28868571e-02 9.33135924e-04]
 [2.83843889e-02 1.65490783e-02 8.69664644e-01 6.80821500e-02]
 [0.00000000e+00 0.00000000e

### On the baseline model with maximum number of 5000 episodes we got around 27% accuracy 

## Baseline Model - On Policy SARSA

### 𝑄 (𝑠,𝑎) ← 𝑄 (𝑠,𝑎) + 𝛼 (𝑟 + 𝛾 𝑄 (𝑠′,𝑎′ )− 𝑄 (𝑠,𝑎) ) , where 𝑎′ is the action, that was taken according to policy 𝜋

In [28]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [29]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [30]:
# Number of episodes agent will play during training 
num_episodes = 5000

# Maximum number of steps agent takes per episode
max_steps_per_episode = 100

#Alpha - Learning Rate
learning_rate = 0.7

#Gamma - Discounting Rate
discount_rate = 0.8

#Epsilon - Exploration Rate
exploration_rate = 1.0

max_exploration_rate = 1
min_exploration_rate = 0.01

# Exponential Decay Rate
exploration_decay_rate = 0.01

In [31]:
# Q-learning algorithm

rewards_all_episodes = []

for episode in range(num_episodes):
    
    # initialize new episode params
    state = env.reset()
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        
        exploration_rate_threshold = random.uniform(0, 1)
        
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()
                
        # Take new action
        new_state, reward, done, info = env.step(action)

        # Update Q-table
        temp_lst = [] # This list holds the value for every action on new state.
        
        for i in range(env.action_space.n):
            temp_lst.append(q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * (q_table[new_state,i])))
        q_table[state, action] = max(temp_lst)
        
        # Set new state
        state = new_state
        rewards_current_episode += reward 
        
        # Add new reward        
        if done == True: 
            break
        
    # Exploration rate decay   
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)

# Calculate and print the average reward per thousand episodes
rewards_per_thosand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/500)
count = 500

In [32]:
print("********Average reward per thousand episodes********\n")
for r in rewards_per_thosand_episodes:
    print(count, ": ", str(sum(r/500)))
    count += 500

********Average reward per thousand episodes********

500 :  0.09000000000000007
1000 :  0.3140000000000002
1500 :  0.33400000000000024
2000 :  0.3140000000000002
2500 :  0.32200000000000023
3000 :  0.2820000000000002
3500 :  0.34000000000000025
4000 :  0.33800000000000024
4500 :  0.34200000000000025
5000 :  0.4240000000000003


In [33]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)    



********Q-table********

[[1.76355638e-03 2.07642054e-03 7.20056865e-03 1.09527494e-03]
 [5.83786640e-04 4.75403974e-04 9.47137511e-04 5.64832348e-03]
 [1.26293343e-03 6.18169003e-03 1.28801045e-03 2.29554626e-04]
 [1.36329256e-04 9.72826692e-06 2.08270708e-05 3.44877577e-03]
 [7.02222568e-02 1.96335682e-03 1.50710803e-03 5.78081030e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.39515805e-05 5.68184090e-05 7.65012200e-03 2.12976384e-07]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.11211900e-03 1.69705765e-03 2.08025770e-03 1.89451161e-01]
 [2.77684173e-02 2.42444552e-01 2.53354569e-03 2.24581963e-03]
 [1.86963279e-01 5.53499735e-03 4.72049624e-03 2.01886641e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.12768279e-03 2.31258729e-02 5.41933152e-01 2.20351748e-02]
 [8.88838899e-02 9.64552568e-02 7.61175576e-01 8.33081346e-02]
 [0.00000000e+00 0.00000000e

### On the On Policy Sarsa with maximum number of 5000 episodes we got around 42 percent accuracy which is around 55 percent increase in from the Off Policy Q - Learning


## Model 2 after tuning hyperparameters 


In [17]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [18]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [19]:
# Number of episodes agent will play during training 
num_episodes = 10000

# Maximum number of steps agent takes per episode
max_steps_per_episode = 100

#Alpha - Learning Rate
learning_rate = 0.1

#Gamma - Discounting Rate
discount_rate = 0.99

#Epsilon - Exploration Rate
exploration_rate = 1

max_exploration_rate = 1
min_exploration_rate = 0.01

# Exponential Decay Rate
exploration_decay_rate = 0.001

In [20]:
# Q-learning algorithm

rewards_all_episodes = []

for episode in range(num_episodes):
    
    # initialize new episode params
    state = env.reset()
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        
        exploration_rate_threshold = random.uniform(0, 1)
        
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()
                
        # Take new action
        new_state, reward, done, info = env.step(action)

        # Update Q-table
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        # Set new state
        state = new_state
        rewards_current_episode += reward 
        
        # Add new reward        
        if done == True: 
            break
        
    # Exploration rate decay   
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)

# Calculate and print the average reward per thousand episodes
rewards_per_thosand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

In [21]:
print("********Average reward per thousand episodes********\n")
for r in rewards_per_thosand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.048000000000000036
2000 :  0.21900000000000017
3000 :  0.3880000000000003
4000 :  0.5350000000000004
5000 :  0.6180000000000004
6000 :  0.6420000000000005
7000 :  0.6840000000000005
8000 :  0.6790000000000005
9000 :  0.6760000000000005
10000 :  0.6790000000000005


In [22]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table) 



********Q-table********

[[0.49949529 0.49753489 0.50123551 0.49851384]
 [0.35179383 0.13085974 0.25594212 0.47515401]
 [0.40864376 0.43497857 0.41378101 0.46077887]
 [0.29818741 0.24431906 0.26375149 0.44408518]
 [0.51574434 0.32715581 0.28182863 0.47735402]
 [0.         0.         0.         0.        ]
 [0.40934045 0.18570822 0.14658024 0.10648827]
 [0.         0.         0.         0.        ]
 [0.3961589  0.47470868 0.39116167 0.5577665 ]
 [0.55517202 0.62366524 0.44584063 0.45073623]
 [0.64778268 0.41894073 0.31832293 0.33636616]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.33607592 0.60998504 0.70204051 0.51451692]
 [0.69275218 0.81161666 0.78475383 0.73208297]
 [0.         0.         0.         0.        ]]


In [23]:
# Watch our agent play Frozen Lake by playing the best action 
# from each state according to the Q-table

for episode in range(3):
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)
    
    
    for step in range(max_steps_per_episode):            
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
    
    
        action = np.argmax(q_table[state,:])        
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
            clear_output(wait=True)
            break          
            
    state = new_state
        
env.close()

  (Right)
SFFF
FHFH
FFFH
HFFG
****You fell through a hole!****


### On the this model with maximum number of 10000 episodes we got around 67% accuracy which around 140% of the baseline model

### Establish a baseline performance. How well did your RL Q-learning do on your problem?

### What are the states, the actions and the size of the Q-table?

### What are the rewards? Why did you choose them?

### How did you choose alpha and gamma in the following equation? 

### Try at least one additional value for alpha and gamma. How did it change the baseline performance?

### Try a policy other than maxQ(s', a'). How did it change the baseline performance?

### How did you choose your decay rate and starting epsilon? Try at least one additional value for epsilon and the decay rate. How did it change the baseline performance? What is the value of epsilon when if you reach the max steps per episode

### What is the average number of steps taken per episode?

### Does Q-learning use value-based or policy-based iteration?

### What is meant by expected lifetime value in the Bellman equation?

### Conclusion

I have successfully implemented Baseline Model with different hyperparameters like Number of Episodes, Max steps per episode, Alpha - Learning Rate, Gamma - Discounting Rate,Epsilon - Exploration Rate, Exploration Decay Rate and found that on baseline model the accuracy was around 27 percent while after tuning the hyperparameters the accuracy increased about  percent which was around 67 percent from baseline model hyperparameters

### Contributions

30% of explanation and code is done by me.

60-70% of resource is from web and citations are given below.

10% of resource is from prof. Nik Brown notes.

### Citations

https://www.biostat.wisc.edu/~craven/cs760/lectures/reinforcement.pdf

http://gym.openai.com/envs/FrozenLake-v0/

https://deeplizard.com/learn/video/QK_PP_2KgGE

https://stats.stackexchange.com/questions/184657/what-is-the-difference-between-off-policy-and-on-policy-learning


### License

#### Author Abhi Patodi

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.